## **Impost necessary liberties** 🔳 

In [ ]:
!pip install tensorflow==1.13.1
#%tensorflow_version 1.x
!pip install --upgrade h5py==2.10.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pytesseract
!sudo apt-get install tesseract-ocr-ind

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-ind is already the newest version (4.00~git24-0e00fe6-1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
!pip install tensorflow==1.13.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
print(tf.__version__)

1.13.1


In [ ]:
!pip install --user --upgrade tensorflow-probability

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import cv2
import numpy as np
import pytesseract
import matplotlib.pyplot as plt
import pandas as pd
from google.colab.patches import cv2_imshow
from PIL import Image
from pytesseract import Output
import cv2
import pytesseract
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

import os
import sys
import random
import math
import re
import time
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import shutil
import zipfile

import glob



# Train MAsk RCNN Model for Card Detection 💪

▶ change the dir to MaskRcnn model dir 

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/Mask-R-CNN/Mask_RCNN-master

/content/drive/MyDrive/Colab Notebooks/Mask-R-CNN/Mask_RCNN-master


▶ Install Keras ver 2.0.8

In [ ]:
!pip install keras==2.0.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


▶ Import MaskRCNN files 

In [ ]:
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log


▶ Helper Functions 

In [ ]:
class CustomDataset(utils.Dataset):

    def load_custom(self, annotation_json, images_dir, dataset_type="train"):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """

        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()


        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']

            class_name = category['name']
            if class_id < 1:
                print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(
                    class_name))
                return

            self.add_class(source_name, class_id, class_name)

        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)

        # Get all images and add them to the dataset
        seen_images = {}

        # Split the dataset, if train, get 90%, else 10%
        len_images = len(coco_json['images'])
        if dataset_type == "train":
            img_range = [int(len_images / 9), len_images]
        else:
            img_range = [0, int(len_images / 9)]

        for i in range(img_range[0], img_range[1]):
            image = coco_json['images'][i]
            image_id = image['id']
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))

                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]

                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )

    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []

        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        #print("Class_ids, ", class_ids)
        return mask, class_ids

    def count_classes(self):
        class_ids = set()
        for image_id in self.image_ids:
            image_info = self.image_info[image_id]
            annotations = image_info['annotations']

            for annotation in annotations:
                class_id = annotation['category_id']
                class_ids.add(class_id)

        class_number = len(class_ids)
        return class_number




        
def load_image_dataset(annotation_path, data_path, dataset_type):
    train_data = CustomDataset()
    train_data.load_custom(annotation_path, data_path, dataset_type)
    train_data.prepare()
    return train_data
class myMaskRCNNConfig(Config):
    # give the configuration a recognizable name
    NAME = "MaskRCNN_config"

    # set the number of GPUs to use along with the number of images
    # per GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # number of classes (we would normally add +1 for the background)
    # fire name + last name + gender + card no + date of birth + BG
    NUM_CLASSES = 1 + 1

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Learning rate
    LEARNING_RATE = 0.001

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.90

    # setting Max ground truth instances
    MAX_GT_INSTANCES = 1
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512



▶ Load the dataset 

In [ ]:



train = load_image_dataset(os.path.join("/content/drive/MyDrive/Colab Notebooks/id_detection/annotation/logo_an.json"), "/content/drive/MyDrive/Colab Notebooks/id_detection/train", "train")
test = load_image_dataset(os.path.join("/content/drive/MyDrive/Colab Notebooks/id_detection/annotation/logo_an.json"), "/content/drive/MyDrive/Colab Notebooks/id_detection/train", "test")
class_number = train.count_classes()
print('Train: %d' % len(train.image_ids))
print('test: %d' % len(test.image_ids))
print("Classes: {}".format(class_number))

Train: 15
test: 1
Classes: 1


▶ train the model


In [ ]:
                                                                     
# save logs and trained model
model_dir= "/content/drive/MyDrive/Colab Notebooks/Mask-R-CNN/Mask_RCNN-master/new_model_28"

# Local path to trained weights file
COCO_MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/mask_rcnn_coco.h5"
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)


config = myMaskRCNNConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=model_dir)
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    print(COCO_MODEL_PATH)
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                        exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)


# train the model
model.train(train, test,
        learning_rate=config.LEARNING_RATE,
        epochs=4,
        layers='heads')



/content/drive/MyDrive/Colab Notebooks/mask_rcnn_coco.h5

Starting at epoch 0. LR=0.001

Checkpoint Path: /content/drive/MyDrive/Colab Notebooks/Mask-R-CNN/Mask_RCNN-master/new_model_28/maskrcnn_config20220902T0955/mask_rcnn_maskrcnn_config_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn

/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. 

  1/100 [..............................] - ETA: 6038s - loss: 8.3870 - rpn_class_loss: 0.0625 - rpn_bbox_loss: 5.0840 - mrcnn_class_loss: 1.2784 - mrcnn_bbox_loss: 1.4880 - mrcnn_mask_loss: 0.4741

/usr/local/lib/python3.7/dist-packages/skimage/transform/_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


KeyboardInterrupt: ignored

## **Card Detection Model 🔍 🎴**

▶ Create configuration

In [ ]:


def load_model():
    config = myMaskRCNNConfig()


    # Recreate the model in inference mode
    model = modellib.MaskRCNN(mode="inference",
                              config=config,
                              model_dir="/content/drive/MyDrive/Colab Notebooks/Mask-R-CNN/Mask_RCNN-master/new_model_28")

    # Get path to saved weights
    # Either set a specific path or find last trained weights
    #model With 2 epochs 
    #model_path = os.path.abspath("/content/drive/MyDrive/Colab Notebooks/Mask-R-CNN/Mask_RCNN-master/new_model_28/maskrcnn_config20220828T1213/mask_rcnn_maskrcnn_config_0001.h5")
    model_path = os.path.abspath("/content/drive/MyDrive/Colab Notebooks/Mask-R-CNN/Mask_RCNN-master/new_model_28/maskrcnn_config20220902T0455/mask_rcnn_maskrcnn_config_0003.h5")
    #model_path = model.find_last()

    # Load trained weights
    print("Loading weights from ", model_path)
    model.load_weights(model_path, by_name=True)
    return model, config

model, inference_config =load_model()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


Loading weights from  /content/drive/MyDrive/Colab Notebooks/Mask-R-CNN/Mask_RCNN-master/new_model_28/maskrcnn_config20220902T0455/mask_rcnn_maskrcnn_config_0003.h5
Re-starting from epoch 3


▶ Test the model  

In [ ]:
import pathlib
import tensorflow as tf
import cv2
import argparse
from google.colab.patches import cv2_imshow
from PIL import Image # Notice the 'from PIL' at the start of the line
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array


In [ ]:

for imagePath in glob.glob("/content/drive/MyDrive/Colab Notebooks/id_detection/test"+ "/*.jpeg"):
    image = cv2.imread(imagePath)
    (h,w)=image.shape[:2]
    img = img_to_array(image)
    # detecting objects in the image
    result= model.detect([img])
    r = result[0]
    Detection_text="Not Detected"
    color = (0, 0, 255)
    class_name="western digital id"
    found = None
    if r["scores"].any():
      color = (125, 246, 55)
      Detection_text="Detected"
      an_array = np.array(r["scores"])
      index = np.argmax(an_array)
      y1, x1, y2, x2=tuple(r["rois"][index])
      #visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], test.class_names, r['scores'],title="Predictions")
      score = round(r["scores"][index], 2)
      text=str(class_name +"  "+str(score))
      cv2.putText(img,text,(x1, y1 - 15),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
      cv2.rectangle(img, (x1, y1), (x2, int(y2*3)), (0, 0, 255), 2) # big bbox
      #cv2.rectangle(img, (x1, y1), (x2, int(y2)), (0, 0, 255), 2)
    cv2.putText(img, Detection_text, (0 + int(w/45),0 + int(h/17.5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    cv2_imshow(img)


## **ID Card detaction** 🆔 🕵

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights


--2022-08-28 17:11:18--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  16.6MB/s    in 16s     

2022-08-28 17:11:35 (14.7 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
classes = None
with open("/content/drive/MyDrive/Colab Notebooks/id_detection/coco.names.txt", "r") as f:
  classes = [line.strip() for line in f.readlines()]
Card_detected="Not Detected"
color = (0, 0, 255)
id_numbers_detected="Not Detected"
color2 = (0, 0, 255)


net = cv2.dnn.readNet('/content/drive/MyDrive/Colab Notebooks/Mask-R-CNN/Mask_RCNN-master/yolov3.weights', '/content/drive/MyDrive/Colab Notebooks/id_detection/yolov3.cfg.txt')
for imagePath in glob.glob("/content/drive/MyDrive/Colab Notebooks/id_detection/test"+ "/*.jpeg"):
    image = cv2.imread(imagePath)
    (h,w)=image.shape[:2]
    img = img_to_array(image)
    # detecting objects in the image
    result= model.detect([img])
    r = result[0]
    print(r)
    class_name="western digital id"
    found = None
    if r["scores"].any():
      Card_detected="Detected"
      color = (125, 246, 55)
      Detection_text="Detected"
      an_array = np.array(r["scores"])
      index = np.argmax(an_array)
      y1, x1, y2, x2=tuple(r["rois"][index])
      logo_imgCrop = image[y1:y2*3, x1:x2]
      cv2_imshow(logo_imgCrop)
      #visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], test.class_names, r['scores'],title="Predictions")
      score = round(r["scores"][index]*100, 2) 
      text=str(class_name +"  "+str(score)+"%")
      cv2.putText(image,text,(x1, y1 - 15),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
     # cv2.rectangle(image, (x1, y1), (x2, int(y2*3)), (0, 0, 255), 2) # big bbox
      #cv2.rectangle(img, (x1, y1), (x2, int(y2)), (0, 0, 255), 2)
      #cv2.putText(image, "ID Card :"+, (0 + int(w/45),0 + int(h/16.5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    
      net.setInput(cv2.dnn.blobFromImage(logo_imgCrop, 0.00392, (416,416), (0,0,0), True, crop=False))
      layer_names = net.getLayerNames()

      #output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
      output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

      outs = net.forward(output_layers)
      class_ids = []
      confidences = []
      boxes = []
      Width = logo_imgCrop.shape[1]
      Height = logo_imgCrop.shape[0]
      
      for out in outs:
          for detection in out:
              scores = detection[5:]
              class_id = np.argmax(scores)
              confidence = scores[class_id]
              if confidence > 0.1:
                  center_x = int(detection[0] * Width)
                  center_y = int(detection[1] * Height)
                  w = int(detection[2] * Width)
                  h = int(detection[3] * Height)
                  x = center_x - w / 2
                  y = center_y - h / 2
                  class_ids.append(class_id)
                  confidences.append(float(confidence))
                  boxes.append([x, y, w, h])
      indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)

      for i in indices:
          box = boxes[i]
          x=round(box[0])
          y=round(box[1])
          w=round(box[2])
          h=round(box[3])
          new_y=round(box[1]*1)
          new_h=round(box[3]*1.75)

          if class_ids[i]==0:
              label = str(classes[class_id]) 
              #cv2.rectangle(image, (x,y), (x+w,y+h), (0, 0, 0), 2)
              #cv2.rectangle(logo_imgCrop, (x,new_y), (x+w+15,y+new_h), (0, 255, 0), 2)

              #cv2.putText(logo_imgCrop, label, (round(box[0])-10,round(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
              imgCrop = logo_imgCrop[y:y + new_h, x:x + w+15]
              #cropped_image = img_cropped[int(y2/3):y2*3, x1:x2]   
              #mgCrop = image[y:y + new_h, x:x + w]
              cv2_imshow(imgCrop)
              ## (1) gray scale crop image 
              gray = cv2.cvtColor(imgCrop, cv2.COLOR_BGR2GRAY)

              ## (2) Threshold
              th, threshed = cv2.threshold(gray, 100, 255, cv2.THRESH_TRUNC)
              cv2_imshow(threshed)
              ## (3) Detect
              result = pytesseract.image_to_string((threshed), lang="ind")
              print(result)
              text_list=[]
              ## (5) Normalize
              for word in result.split("\n"):
                if "”—" in word:
                  word = word.replace("”—", ":")
                
                #normalize NIK
                if "NIK" in word:
                  nik_char = word.split()
                  if "D" in word:
                    word = word.replace("D", "0")
                  if "?" in word:
                    word = word.replace("?", "7") 
                print(word)
                word_len=len(word)
                print(word_len)
                if word.isnumeric() and word_len > 4 :
                  id_numbers_detected="Detected"
                  color2 = (125, 246, 55)
                  detected_num =word
                  cv2.putText(image, detected_num, (0 + int(Width/45),0 + int(Height/6.5)), cv2.FONT_HERSHEY_SIMPLEX, 1, (125, 246, 55), 2, cv2.LINE_AA)

                  print("Number detected")
                else :
                  
                  print("No Number detected")


  #print(word)
#print(text_list)
    cv2.putText(image, "ID Card: "+Card_detected, (0 + int(Width/45),0 + int(Height/22.5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    cv2.putText(image, "ID Number: "+id_numbers_detected, (0 + int(Width/45),0 + int(Height/9.5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color2, 2, cv2.LINE_AA)

    cv2_imshow(image)
  

▶ view the video 

In [ ]:

from IPython.display import HTML
from base64 import b64encode
video = '/content/drive/MyDrive/Colab Notebooks/id_detection/WIN_20220830_08_48_07_Pro.mp4'  

video_path = video ######## put the name of your video file HERE

mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

▶ video detection

In [ ]:
classes = None
with open("/content/drive/MyDrive/Colab Notebooks/id_detection/coco.names.txt", "r") as f:
  classes = [line.strip() for line in f.readlines()]

# Video 
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
video = '/content/drive/MyDrive/Colab Notebooks/id_detection/WIN_20220901_12_59_50_Pro.mp4'  
start = time.time()
cap = cv2.VideoCapture(video)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("Video resolution : ", height, width)
assert cap.isOpened(), 'Cannot capture source'
#Add extension mask_rcnn to video name
video_detection_path =  video.split(".")[0] + "_id_detection_vid2." + video.split(".")[1]


#Video save
videoSaved = cv2.VideoWriter(video_detection_path, apiPreference=cv2.CAP_FFMPEG,
                             fourcc = fourcc,fps =20, frameSize=(width, height),	
                             isColor= True)
frames = 0
net = cv2.dnn.readNet('/content/drive/MyDrive/Colab Notebooks/Mask-R-CNN/Mask_RCNN-master/yolov3.weights', '/content/drive/MyDrive/Colab Notebooks/id_detection/yolov3.cfg.txt')

while cap.isOpened():
    Card_detected="Not Detected"
    color = (0, 0, 255)
    id_numbers_detected="Not Detected"
    color2 = (0, 0, 255)
    ret, frame = cap.read()
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#for imagePath in glob.glob("/content/drive/MyDrive/Colab Notebooks/id_detection/test"+ "/*.jpeg"):
  #  image = cv2.imread(imagePath)
    #(h,w)=image.shape[:2]
    if ret:    
        img = img_to_array(frame)
        # detecting objects in the image
        result= model.detect([img])
        r = result[0]
        #print(r)
        class_name="western digital id"
        found = None
        if r["scores"].any():

          Detection_text="Detected"
          an_array = np.array(r["scores"])
          index = np.argmax(an_array)
          y1, x1, y2, x2=tuple(r["rois"][index])
          logo_imgCrop = frame[y1:y2*3, x1:x2]
          #cv2_imshow(logo_imgCrop)
          #visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], test.class_names, r['scores'],title="Predictions")
          score = round(r["scores"][index]*100, 2) 
          #if score >= 85:
          Card_detected="Detected"
          color = (125, 246, 55)
          text=str(class_name +"  "+str(score)+"%")
          cv2.putText(frame,text,(x1, y1 - 8), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.9, (255, 0, 0), 1,cv2.LINE_AA)
          cv2.rectangle(frame, (x1, y1), (x2, int(y2*3)), (255, 0, 0), 2) # big bbox
          #cv2.rectangle(img, (x1, y1), (x2, int(y2)), (0, 0, 255), 2)
          #cv2.putText(image, "ID Card :"+, (0 + int(w/45),0 + int(h/16.5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
        
          net.setInput(cv2.dnn.blobFromImage(logo_imgCrop, 0.00392, (416,416), (0,0,0), True, crop=False))
          layer_names = net.getLayerNames()

          #output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
          output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

          outs = net.forward(output_layers)
          class_ids = []
          confidences = []
          boxes = []
          Width_ = logo_imgCrop.shape[1]
          Height_ = logo_imgCrop.shape[0]
          
          for out in outs:
              for detection in out:
                  scores = detection[5:]
                  class_id = np.argmax(scores)
                  confidence = scores[class_id]
                  if confidence > 0.1:
                      center_x = int(detection[0] * Width_)
                      center_y = int(detection[1] * Height_)
                      w = int(detection[2] * Width_)
                      h = int(detection[3] * Height_)
                      x = center_x - w / 2
                      y = center_y - h / 2
                      class_ids.append(class_id)
                      confidences.append(float(confidence))
                      boxes.append([x, y, w, h])
          indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)
          if len(indices) > 0 :
              for i in indices:
                  box = boxes[i]
                  x=round(box[0])
                  y=round(box[1])
                  w=round(box[2])
                  h=round(box[3])
                  if x  >= 0 :
                    new_y=round(box[1]*1)
                    new_h=round(box[3]*1.75)

                    if class_ids[i]==0:
                        label = str(classes[class_id]) 
                        #cv2.rectangle(image, (x,y), (x+w,y+h), (0, 0, 0), 2)
                       # cv2.rectangle(logo_imgCrop, (x,new_y), (x+w+15,y+new_h), (0, 255, 0), 2)

                        #cv2.putText(logo_imgCrop, label, (round(box[0])-10,round(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
                        imgCrop = logo_imgCrop[y:y + new_h, x:x + w+15]
                        #cropped_image = img_cropped[int(y2/3):y2*3, x1:x2]   
                        #mgCrop = image[y:y + new_h, x:x + w]
                        #cv2_imshow(imgCrop)
                        ## (1) gray scale crop image 
                        gray = cv2.cvtColor(imgCrop, cv2.COLOR_BGR2GRAY)
                        #cv2_imshow(gray)

                        ## (2) Threshold
                        th, threshed = cv2.threshold(gray, 80, 255, cv2.THRESH_TRUNC)
                        ## (3) Detect
                        result = pytesseract.image_to_string((gray), lang="ind")
                        #print(result)
                        text_list=[]
                        ## (5) Normalize
                        for word in result.split("\n"):
                          if "”—" in word:
                            word = word.replace("”—", ":")
                          
                          #normalize NIK
                          if "NIK" in word:
                            nik_char = word.split()
                            if "D" in word:
                              word = word.replace("D", "0")
                            if "?" in word:
                              word = word.replace("?", "7") 
                         # print(word)
                          word_len=len(word)
                          #print(word_len)
                          if word.isnumeric() and word_len > 4 :
                            id_numbers_detected="Detected"
                            color2 = (125, 246, 55)
                            detected_num =word
                            #cv2.putText(frame,detected_num,(x1, y1 - 15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.9, (255, 0, 0), 1,cv2.LINE_AA)

                            cv2.putText(frame, detected_num, (0 + int(width/45),0 + int(height/1.3)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.9, (125, 246, 55), 1,cv2.LINE_AA)

                          # print("Number detected")
                        #  else :
                            
                          #  print("No Number detected")
                  else :
                      continue    


      #print(word)
    #print(text_list)
       # scale = 1 # this value can be from 0 to 1 (0,1] to change the size of the text relative to the image
       # fontScale = min(imageWidth,imageHeight)/(25/scale)
        cv2.putText(frame, "ID Card: "+Card_detected, (0 + int(width/45),0 + int(height/1.02)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.9, color, 1,cv2.LINE_AA)
        cv2.putText(frame, "ID Number: "+id_numbers_detected, (0 + int(width/45),0 + int(height/1.1)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.9, color2, 1,cv2.LINE_AA)
        #cv2_imshow(frame)
          # video save line
        videoSaved.write(frame)
          
        frames += 1
        if frames%50 == 0: print("{} processed frames".format(frames))       
    else :
        break
videoSaved.release()
end = time.time()
print("Videosaved as " + video_detection_path)
print('Total time elapsed: {:.3f} segundos'.format((end - start)))

Video resolution :  288 352
50 processed frames
100 processed frames
150 processed frames
200 processed frames
250 processed frames
300 processed frames
350 processed frames
400 processed frames
450 processed frames
500 processed frames
550 processed frames
600 processed frames
650 processed frames
700 processed frames
750 processed frames
800 processed frames
850 processed frames
900 processed frames
950 processed frames
1000 processed frames
1050 processed frames
1100 processed frames
1150 processed frames
1200 processed frames
1250 processed frames
1300 processed frames
1350 processed frames
1400 processed frames
1450 processed frames
1500 processed frames
Videosaved as /content/drive/MyDrive/Colab Notebooks/id_detection/WIN_20220901_12_59_50_Pro_id_detection_vid2.mp4
Total time elapsed: 14898.349 segundos
